In [168]:
import asyncio
import datetime
from datetime import time, timedelta
from dateutil import tz
import aiohttp
import pandas as pd

In [169]:
current_timezone = tz.gettz("Asia/Singapore")

In [170]:
headers = {'content-type':"application/json"}
payload = {'date_time':"2024-05-21T00:00:06"}

In [171]:
result = requests.get(url='https://api.data.gov.sg/v1/transport/taxi-availability', headers=headers, params=payload)

In [172]:
result.raise_for_status()

In [173]:
result.text

'{"type":"FeatureCollection","crs":{"type":"link","properties":{"href":"http://spatialreference.org/ref/epsg/4326/ogcwkt/","type":"ogcwkt"}},"features":[{"type":"Feature","geometry":{"type":"MultiPoint","coordinates":[[103.62317,1.282],[103.62535,1.29009],[103.6358,1.32427],[103.64903,1.32418],[103.65209,1.31699],[103.65256,1.31738],[103.66275,1.31264],[103.66544,1.30695],[103.66937,1.32559],[103.67429,1.3242],[103.67676,1.32663],[103.68578,1.35047],[103.68597,1.32966],[103.68682385,1.34046528333333],[103.68875,1.34],[103.688852166667,1.34286033333333],[103.688957166667,1.340972],[103.69020795,1.35083663333333],[103.69059,1.34175],[103.69069,1.34],[103.69076,1.34],[103.6908,1.31577],[103.69169,1.31535],[103.69309,1.33878],[103.69339,1.34],[103.69344,1.3441],[103.69521,1.31965],[103.695886716667,1.34128811666667],[103.696014666667,1.3452675],[103.69619,1.33472],[103.69627,1.31723],[103.69636,1.34728],[103.69745,1.31001],[103.69791725,1.3412111],[103.69797,1.32793],[103.69848,1.37268],[1

In [175]:
async def get_interval(date_time: str):

    if len(date_time) == 16:
        process_date_time = datetime.datetime.strptime(date_time, "%Y-%m-%d %H:%M")
        start_date_time = process_date_time - timedelta(minutes=5)
        end_date_time = process_date_time
        intervals = [
            dttm.strftime("%Y-%m-%dT%H:%M:%S")
            for dttm in pd.date_range(start_date_time, end_date_time, freq="1min", inclusive="right")
        ]

    else:
        current_date_time = datetime.datetime.now(datetime.UTC).astimezone(current_timezone)

        if date_time == "today":
            process_date = datetime.datetime.now(datetime.UTC).astimezone(current_timezone).date()
        else:
            process_date = datetime.datetime.strptime(date_time, "%Y-%m-%d").date()

        start_date_time = datetime.datetime.combine(process_date, time.min)

        if process_date < current_date_time.date():
            end_date_time = datetime.datetime.combine(process_date, time.max)
        else:
            end_date_time = current_date_time.replace(tzinfo=None)

        intervals = [
            dttm.strftime("%Y-%m-%dT%H:%M:%S") for dttm in pd.date_range(start_date_time, end_date_time, freq="1min")
        ]

    return intervals

In [182]:
data = await get_interval(date_time='2024-05-21')

In [184]:
len(data)

1440